In [1]:
import re
from pathlib import Path
import numpy as np
from pprint import pprint
from builder.file_manager import *
from builder.spell import build_spellbook
from sympy import flatten

In [2]:
pdf_path = Path("Feuille de personnage Remaster (éditable) 1.1.pdf")
json_path = Path("Persolv3.json")
output_path = Path("feuille_remplie.pdf")

In [3]:
# fonctions
def build_abilities(character_data):
    """Construit les statistiques de base d'un personnage avant d'appliquer les modifications."""

    def abilities_from_item(abilities, item_str):
        item_next = next(
            (item for item in character_data["items"] if item["type"] == item_str), None
        )
        if item_next:
            for boost in item_next["system"]["boosts"].values():
                if boost.get("selected"):
                    abilities[boost.get("selected")]["value"] += 2
                else:
                    for ability in boost["value"]:
                        abilities[ability]["value"] += 2

        # Appliquer les malus d'ascendance
        if item_next:
            if item_next["system"].get("flaws"):
                for flaw in item_next["system"]["flaws"].values():
                    if flaw.get("value"):
                        for ability in flaw["value"]:
                            abilities[ability]["value"] -= 2
        return abilities

    def bump_ability(ability):
        """Augmente les capacités d'un personnage en fonction de son niveau."""
        if ability["value"] >= 18:
            if ability["partial"]:
                ability["value"] += 2
                ability["partial"] = False
            else:
                ability["partial"] = True
        else:
            ability["value"] += 2
        return ability

    # Récupérer le niveau
    level = character_data["system"]["details"]["level"]["value"]

    # Construire les caractéristiques de base
    base_abilities = {
        "str": {"value": 10, "mod": 0, "partial": False},
        "dex": {"value": 10, "mod": 0, "partial": False},
        "con": {"value": 10, "mod": 0, "partial": False},
        "int": {"value": 10, "mod": 0, "partial": False},
        "wis": {"value": 10, "mod": 0, "partial": False},
        "cha": {"value": 10, "mod": 0, "partial": False},
    }

    # Appliquer les bonus d'ascendance
    abilities = abilities_from_item(base_abilities, "ancestry")

    # Appliquer les bonus d'historique
    abilities = abilities_from_item(abilities, "background")

    # Appliquer les bonus de classe
    class_item = next(
        (item for item in character_data["items"] if item["type"] == "class"), None
    )
    if class_item and class_item["system"]["keyAbility"]["selected"]:
        abilities[class_item["system"]["keyAbility"]["selected"]]["value"] += 2

    # 4. Appliquer les augmentations de niveaux
    level_boosts = character_data["system"]["build"]["attributes"]["boosts"]
    for level_unlocked, level_boost in level_boosts.items():
        if int(level_unlocked) <= int(level):
            for ability in level_boost:
                abilities[ability] = bump_ability(abilities[ability])
        elif int(level_unlocked) > int(level) & int(level_unlocked) < int(level) + 5:
            borne_haute = int(level) - (int(level_unlocked) - 5) - 1
            for ability in level_boost[:borne_haute]:
                abilities[ability] = bump_ability(abilities[ability])

    # Calculer les modificateurs
    for ability in base_abilities.values():
        ability["mod"] = (ability["value"] - 10) // 2

    return abilities


def build_competences(character_data, level, abilities):
    def build_competences_rank(character_data):
        def ajouter_si_absente(dico, element):
            if element not in dico.keys():
                dico[element] = {"rank": 1}

        # Compétences choisies par l'utilisateur
        skills_rank = character_data["system"]["skills"]

        # Compétence Donner par les items
        for i in character_data["items"]:
            if i["system"].get("trainedSkills"):
                for skill in i["system"]["trainedSkills"]["value"]:
                    ajouter_si_absente(skills_rank, skill)

        # Compétence donnée par des règles Spécifique à certains items
        for item in character_data["items"]:
            for rule in item["system"]["rules"]:
                if rule.get("flag"):
                    if rule["flag"] == "skill":
                        ajouter_si_absente(skills_rank, rule["selection"])

        return skills_rank

    def calculate_proficiency(rank, level):
        """Calcule le bonus de maîtrise basé sur le rang et le niveau."""
        if rank > 0:
            return level + (rank * 2)
        return 0

    skills = {
        "acrobatics": {
            "rank": 0,
            "mod": 0,
            "ability": "dex",
            "proficiency_bonus": 0,
        },
        "arcana": {
            "rank": 0,
            "mod": 0,
            "ability": "int",
            "proficiency_bonus": 0,
        },
        "athletics": {
            "rank": 0,
            "mod": 0,
            "ability": "str",
            "proficiency_bonus": 0,
        },
        "crafting": {
            "rank": 0,
            "mod": 0,
            "ability": "int",
            "proficiency_bonus": 0,
        },
        "deception": {
            "rank": 0,
            "mod": 0,
            "ability": "cha",
            "proficiency_bonus": 0,
        },
        "diplomacy": {
            "rank": 0,
            "mod": 0,
            "ability": "cha",
            "proficiency_bonus": 0,
        },
        "intimidation": {
            "rank": 0,
            "mod": 0,
            "ability": "cha",
            "proficiency_bonus": 0,
        },
        "medicine": {
            "rank": 0,
            "mod": 0,
            "ability": "wis",
            "proficiency_bonus": 0,
        },
        "nature": {
            "rank": 0,
            "mod": 0,
            "ability": "wis",
            "proficiency_bonus": 0,
        },
        "occultism": {
            "rank": 0,
            "mod": 0,
            "ability": "int",
            "proficiency_bonus": 0,
        },
        "performance": {
            "rank": 0,
            "mod": 0,
            "ability": "cha",
            "proficiency_bonus": 0,
        },
        "religion": {
            "rank": 0,
            "mod": 0,
            "ability": "wis",
            "proficiency_bonus": 0,
        },
        "society": {
            "rank": 0,
            "mod": 0,
            "ability": "int",
            "proficiency_bonus": 0,
        },
        "stealth": {
            "rank": 0,
            "mod": 0,
            "ability": "dex",
            "proficiency_bonus": 0,
        },
        "survival": {
            "rank": 0,
            "mod": 0,
            "ability": "wis",
            "proficiency_bonus": 0,
        },
        "thievery": {
            "rank": 0,
            "mod": 0,
            "ability": "dex",
            "proficiency_bonus": 0,
        },
    }

    ranks = build_competences_rank(character_data)
    # Récupérer les rangs depuis le système
    for skill_name in skills.keys():
        ability = skills[skill_name]["ability"]
        ability_mod = abilities[ability]["mod"]
        rank = ranks.get(skill_name, {"rank": 0})["rank"]
        proficiency_bonus = calculate_proficiency(rank, level)
        skills[skill_name].update(
            {
                "rank": rank,
                "proficiency_bonus": proficiency_bonus,
                "mod": ability_mod + proficiency_bonus,
                "ability_mod": ability_mod,
            }
        )

    return skills


def split_text_so_longer(text, lim_len):
    count = 0
    text_lines = re.split("\n", text)
    if len(text_lines) > 1:
        raise ValueError("Le texte ne doit pas contenir de sauts de ligne.")
    last_line = text_lines[-1]
    while len(last_line) > lim_len and count < 10:
        count += 1
        # Trouver l'espace le plus proche du milieu pour couper le texte
        split_index = last_line.rfind(" ", 0, lim_len) + 1
        if split_index == 0:  # Si aucun espace n'est trouvé, couper au milieu
            split_index = lim_len
        last_line = last_line[:split_index] + "\n" + last_line[split_index:]
        text = "\n".join(text_lines[:-1] + [last_line])
        text_lines = re.split("\n", text)
        last_line = text_lines[-1]
    return text, count


def build_proficiencies(character_data):
    """Construit la liste des compétences et de leurs niveaux de maîtrise."""
    proficiencies = {}
    # Récupérer les compétences depuis les données du personnage
    for skill_name, skill_data in character_data["system"]["skills"].items():
        proficiencies[skill_name] = skill_data["rank"]
    # Récupération des jets de sauvegarde
    proficiencies.update(
        next_item_by_type(character_data, "class")["system"]["savingThrows"]
    )
    # Récupération des attaques
    proficiencies.update(
        next_item_by_type(character_data, "class")["system"]["attacks"]
    )
    # Récupération des defenses
    proficiencies.update(
        next_item_by_type(character_data, "class")["system"]["defenses"]
    )
    # Récupération de la perception
    proficiencies["perception"] = next_item_by_type(character_data, "class")["system"][
        "perception"
    ]
    # Récupération de l'incantation
    if next_item_by_type(character_data, "class")["system"].get("spellcasting", False):
        proficiencies["spellcasting"] = next_item_by_type(character_data, "class")[
            "system"
        ]["spellcasting"]

    # parcourir les items pour upgrade les maitrises
    for act in character_data["items"]:
        if act["system"].get("subfeatures"):
            if act["system"]["subfeatures"].get("proficiencies", False):
                for prof_name, prof_values in act["system"]["subfeatures"][
                    "proficiencies"
                ].items():
                    if proficiencies.get(prof_name, False):
                        proficiencies[prof_name] = max(
                            proficiencies[prof_name], prof_values["rank"]
                        )

                    else:
                        proficiencies[prof_name] = prof_values["rank"]
    return proficiencies


def get_all_languages(character_data):
    """Récupère toutes les langues que parle le personnage."""
    # Dictionnaire pour traduire les codes en noms complets
    language_map = {
        "common": "Commun",
        "elven": "Elfique",
        "gnomish": "Gnome",
        "goblin": "Gobelin",
        "dwarven": "Nain",
        "fey": "Féerique",
        "draconic": "Draconique",
        "empyrean": "Empyréen",
        "jotun": "Jotun",
        "sylvan": "Sylvestre",
        "celestial": "Céleste",
        "infernal": "Infernal",
        "abyssal": "Abyssal",
        "undercommon": "Sous-commun",
        "aklo": "Aklo",
        "aquan": "Aquatique",
        "auran": "Aérien",
        "ignan": "Igné",
        "terran": "Terreux",
        "necril": "Nécril",
        "shadowtongue": "Langue de l'Ombre",
        "orc": "Orque",
        # Ajoutez d'autres langues selon les besoins
    }

    all_languages = set()

    # 1. Langues directement sur le personnage
    if character_data["system"]["details"]["languages"].get("value"):
        for lang in character_data["system"]["details"]["languages"]["value"]:
            all_languages.add(lang)

    # 2. Langues de l'ascendance
    ancestry_item = next_item_by_type(character_data, "ancestry")
    if ancestry_item and ancestry_item["system"].get("languages"):
        # Langues automatiques de l'ascendance
        if ancestry_item["system"]["languages"].get("value"):
            for lang in ancestry_item["system"]["languages"]["value"]:
                all_languages.add(lang)

        # Langues supplémentaires basées sur l'INT
        if ancestry_item["system"]["languages"].get("custom"):
            all_languages.add(ancestry_item["system"]["languages"]["custom"])

    # Convertir les codes en noms complets
    language_names = [
        language_map.get(lang, lang.capitalize()) for lang in all_languages
    ]

    # Trier alphabétiquement pour une présentation cohérente
    language_names.sort()

    return language_names


def calculate_proficiency(rank, level):
    """Calcule le bonus de maîtrise basé sur le rang et le niveau."""
    if rank > 0:
        return level + (rank * 2)
    return 0


# Lister les sorts dans un format adapté au PDF
def lister_sorts(spells, len_max):
    names = []
    actions = []
    for spell in spells:
        spell_name, lines = split_text_so_longer(spell.name, len_max)
        names.append(spell_name)
        actions.append(str(spell.actions if spell.actions else "—") + ("\n" * lines))
    names = "\n".join(names)
    actions = "\n".join(actions)
    return names, actions


def prepare_spells_fields(
    all_fields_dict, character_data, spellcasting_proficiency, abilities
):
    """Remplit les champs de sorts dans le PDF."""

    # Construire le grimoire
    spellbook = build_spellbook(character_data)

    # 1. Tours de magie (cantrips)
    if spellbook["cantrips"]:
        # Déterminer le rang des tours de magie (généralement égal au niveau du personnage/2)
        level = int(character_data["system"]["details"]["level"]["value"])
        cantrip_rank = int(np.ceil(level / 2))

        all_fields_dict["TOURS_DE_MAGIE_RANG"] = cantrip_rank

        # Lister les tours de magie
        (
            all_fields_dict["TOURS_DE_MAGIE_NOM"],
            all_fields_dict["TOURS_DE_MAGIE_ACTIONS"],
        ) = lister_sorts(spellbook["cantrips"], 34)

        # TODO Récupérer le nombre de tours de magie préparées par jour si applicable
        all_fields_dict["TOURS_DE_MAGIE_PAR_JOUR"] = "∞"  # Tours de magie illimités

    # 2. Sorts focalisés
    if spellbook["focus"]:
        # Déterminer le rang des sorts focalisés
        focus_rank = cantrip_rank

        # Points de focalisation (généralement 1, peut être plus avec des dons)
        # À ajuster selon les dons/capacités du personnage
        focus_points = character_data["system"]["resources"]["focus"][
            "value"
        ]  # TODO tester
        for i in range(focus_points):
            all_fields_dict[f"POINT_FOCALISATION_{i+1}"] = "/Oui"

        all_fields_dict["SORTS_FOCALISES_RANG"] = focus_rank

        # Lister les sorts focalisés
        (
            all_fields_dict["SORTS_FOCALISES_NOM"],
            all_fields_dict["SORTS_FOCALISES_ACTIONS"],
        ) = lister_sorts(spellbook["focus"], 34)

    # 3. Sorts réguliers par niveau
    # Sorts réguliers lister
    all_spells, all_actions = lister_sorts(flatten(spellbook["spells"].values()), 22)
    nb_lines = all_spells.count("\n") + 1
    if nb_lines < 60:
        all_fields_dict[f"SORTS_1_NOM"] = all_spells
        all_fields_dict[f"SORTS_1_ACTIONS"] = all_actions
    elif nb_lines < 120:
        all_fields_dict[f"SORTS_1_NOM"] = "\n".join(all_spells[:60])
        all_fields_dict[f"SORTS_1_ACTIONS"] = "\n".join(all_actions[:60])

        all_fields_dict[f"SORTS_2_NOM"] = "\n".join(all_spells[60:120])
        all_fields_dict[f"SORTS_2_ACTIONS"] = "\n".join(all_actions[60:120])
    else:
        all_fields_dict[f"SORTS_1_NOM"] = "\n".join(all_spells[:60])
        all_fields_dict[f"SORTS_1_ACTIONS"] = "\n".join(all_actions[:60])

        all_fields_dict[f"SORTS_2_NOM"] = "\n".join(all_spells[60:119]) + "\n..."
        all_fields_dict[f"SORTS_2_ACTIONS"] = "\n".join(all_actions[60:119]) + "\n..."

    # Trouver les entrées d'incantation
    spellcasting_principal = [
        item for item in character_data["items"] if item["type"] == "spellcastingEntry"
    ][0]
    # Récupérer les emplacements de sorts par niveau
    for slotx, value in spellcasting_principal["system"].get("slots", {}).items():
        if slotx.startswith("slot") and slotx != "slot0":
            spell_slots = value.get("max", 0)
            if spell_slots > 0:
                all_fields_dict[f"EMPLACEMENTS_{slotx[-1]}_PAR_JOUR"] = spell_slots

    # 4. Bonus d'attaque et DD des sorts
    # Déterminer la caractéristique d'incantation et le niveau de maîtrise
    spellcasting_ability = spellcasting_principal["system"]["ability"]["value"]

    # Calculer les bonus
    level = int(character_data["system"]["details"]["level"]["value"])
    ability_mod = abilities[spellcasting_ability]["mod"]
    proficiency_bonus = calculate_proficiency(spellcasting_proficiency, level)

    # Attaque de sort = mod_carac + maîtrise
    spell_attack = ability_mod + proficiency_bonus
    # DD de sort = 10 + mod_carac + maîtrise
    spell_dc = 10 + ability_mod + proficiency_bonus

    all_fields_dict["SORTS_BONUS_ATTAQUE"] = spell_attack
    all_fields_dict["SORTS_DD"] = spell_dc
    all_fields_dict["SORTS_BONUS_ATTAQUE_ESSENTIELLE"] = ability_mod
    all_fields_dict["SORTS_BONUS_ATTAQUE_MAITRISE"] = proficiency_bonus
    all_fields_dict["SORTS_DD_ESSENTIELLE"] = ability_mod
    all_fields_dict["SORTS_DD_MAITRISE"] = proficiency_bonus

    # Cocher le niveau de maîtrise d'incantation
    all_fields_dict["SORTS_DD_QUALIFIE"] = (
        "/Oui" if spellcasting_proficiency >= 1 else "/Off"
    )
    all_fields_dict["SORTS_DD_EXPERT"] = (
        "/Oui" if spellcasting_proficiency >= 2 else "/Off"
    )
    all_fields_dict["SORTS_DD_MAITRE"] = (
        "/Oui" if spellcasting_proficiency >= 3 else "/Off"
    )
    all_fields_dict["SORTS_DD_LEGENDAIRE"] = (
        "/Oui" if spellcasting_proficiency >= 4 else "/Off"
    )

    all_fields_dict["SORTS_BONUS_ATTAQUE_QUALIFIE"] = (
        "/Oui" if spellcasting_proficiency >= 1 else "/Off"
    )
    all_fields_dict["SORTS_BONUS_ATTAQUE_EXPERT"] = (
        "/Oui" if spellcasting_proficiency >= 2 else "/Off"
    )
    all_fields_dict["SORTS_BONUS_ATTAQUE_MAITRE"] = (
        "/Oui" if spellcasting_proficiency >= 3 else "/Off"
    )
    all_fields_dict["SORTS_BONUS_ATTAQUE_LEGENDAIRE"] = (
        "/Oui" if spellcasting_proficiency >= 4 else "/Off"
    )

    # 5. Type d'incantateur (préparé ou spontané)
    # Rechercher dans les entrées d'incantation - CORRECTION
    spellcasting_type = spellcasting_principal["system"]["prepared"]["value"]

    if spellcasting_type == "prepared":
        all_fields_dict["INCANTATEUR_PREPARE"] = "/Oui"
    else:
        all_fields_dict["INCANTATEUR_SPONTANE"] = "/Oui"

    # 6. Tradition d'incantation
    # Rechercher dans les entrées d'incantation
    tradition = spellcasting_principal["system"]["tradition"]["value"]

    if tradition:
        if tradition == "arcane":
            all_fields_dict["TRADITION_ARCANIQUE"] = "/Oui"
        elif tradition == "divine":
            all_fields_dict["TRADITION_DIVINE"] = "/Oui"
        elif tradition == "occult":
            all_fields_dict["TRADITION_OCCULTE"] = "/Oui"
        elif tradition == "primal":
            all_fields_dict["TRADITION_PRIMORDIALE"] = "/Oui"


def list_don_by_categ(character_data, categ):
    return [item for item in character_data["items"] if item["type"] == categ]


def next_item_by_type(character, item_type):
    return next(
        (item for item in character["items"] if item["type"] == item_type), None
    )


def description_text_cleaner(text):
    text = re.sub(r"@UUID\[.*?\]{(.+?)}", r"\1", text)
    text = re.sub(r"<strong>(.+?)</strong>", r"\1:", text)
    text = re.sub(r"<hr />", "", text)
    text = re.sub(r"<[^>]+>", "", text)
    return text

In [4]:
# Chargement des données
character = load_character_data(json_path)

In [5]:
# Initialisation du PDF
reader, writer = init_pdf_writer(pdf_path)

### construction des states

In [6]:
all_fields_dict = {field_name: None for field_name in reader.get_fields()}

In [7]:
def prepare_pdf_fields(all_fields_dict, character_data):
    """Prépare les champs PDF à partir du dictionnaire de tous les champs.

    Args:
        writer (PdfWriter): Le writer PDF
        all_fields_dict (dict): Dictionnaire contenant tous les champs et leurs valeurs
    """
    # Récupérer le niveau du personnage
    level = int(character_data["system"]["details"]["level"]["value"])

    # Informations de base
    all_fields_dict["NOM_PERSONNAGE"] = character_data["name"]
    all_fields_dict["NIVEAU"] = str(level)
    all_fields_dict["ASCENDANCE"] = next_item_by_type(character_data, "ancestry")[
        "name"
    ]
    all_fields_dict["HERITAGE_TRAITS"] = next_item_by_type(character_data, "heritage")[
        "name"
    ]
    all_fields_dict["HISTORIQUE"] = next_item_by_type(character_data, "background")[
        "name"
    ]
    all_fields_dict["CLASSE"] = next_item_by_type(character_data, "class")["name"]

    # Attributs
    ATTRIBUTES = {
        "str": "FORCE",
        "dex": "DEXTERITE",
        "con": "CONSTITUTION",
        "int": "INTELLIGENCE",
        "wis": "SAGESSE",
        "cha": "CHARISME",
    }
    atrs_char = build_abilities(character_data)

    for atr_char, atr_pdf in ATTRIBUTES.items():
        all_fields_dict[f"{atr_pdf}"] = atrs_char[atr_char]["mod"]
        all_fields_dict[f"{atr_pdf}_PRIME_PARTIELLE"] = (
            "/Oui" if atrs_char[atr_char]["partial"] else "/Off"
        )

    # PV
    all_fields_dict["PV_MAXIMUM"] = (
        (
            next_item_by_type(character_data, "class")["system"]["hp"]
            + atrs_char["con"]["mod"]
        )
        * level
    ) + next_item_by_type(character_data, "ancestry")["system"]["hp"]

    # Bonus de maitrise
    proficiency_rank = build_proficiencies(character_data)

    # Compétences
    COMPETENCES = {
        "acrobatics": "ACROBATIES",
        "arcana": "ARCANES",
        "athletics": "ATHLETISME",
        "crafting": "ARTISANAT",
        "deception": "DUPERIE",
        "diplomacy": "DIPLOMATIE",
        "intimidation": "INTIMIDATION",
        "medicine": "MEDECINE",
        "nature": "NATURE",
        "occultism": "OCCULTISME",
        "performance": "REPRESENTATION",
        "religion": "RELIGION",
        "society": "SOCIETE",
        "stealth": "DISCRETION",
        "survival": "SURVIE",
        "thievery": "LARCIN",
    }
    compts_char = build_competences(character_data, level, abilities=atrs_char)

    for compt_char, compt_pdf in COMPETENCES.items():
        all_fields_dict[f"{compt_pdf}"] = compts_char[compt_char]["mod"]
        all_fields_dict[f"{compt_pdf}_QUALIFIE"] = (
            "/Oui" if compts_char[compt_char]["rank"] >= 1 else "/Off"
        )
        all_fields_dict[f"{compt_pdf}_EXPERT"] = (
            "/Oui" if compts_char[compt_char]["rank"] >= 2 else "/Off"
        )
        all_fields_dict[f"{compt_pdf}_MAITRE"] = (
            "/Oui" if compts_char[compt_char]["rank"] >= 3 else "/Off"
        )
        all_fields_dict[f"{compt_pdf}_LEGENDAIRE"] = (
            "/Oui" if compts_char[compt_char]["rank"] >= 4 else "/Off"
        )
        for ability in ATTRIBUTES.values():
            if f"{compt_pdf}_{ability}" in all_fields_dict.keys():
                all_fields_dict[f"{compt_pdf}_{ability}"] = compts_char[compt_char][
                    "ability_mod"
                ]
        all_fields_dict[f"{compt_pdf}_MAITRISE"] = compts_char[compt_char][
            "proficiency_bonus"
        ]
        # TODO avoir si il est possible de déduire une logique particulière de la construction des objets et leur apport de statistiques
        # all_fields_dict[f"{compt_pdf}_OBJET"] =
        # all_fields_dict[f"{compt_pdf}_ARMURE"] =

    # Sauvegardes
    SAVES = {
        "fortitude": {"name": "REFLEXES", "ability": "con"},
        "reflex": {"name": "VIGUEUR", "ability": "dex"},
        "will": {"name": "VOLONTE", "ability": "wis"},
    }

    saves_data = {
        k: v
        for k, v in proficiency_rank.items()
        if k in ["fortitude", "reflex", "will"]
    }

    for save_key, save_info in SAVES.items():
        rank = saves_data[save_key]
        save_pdf = save_info["name"]
        ability = save_info["ability"]

        # Calculer le modificateur total de sauvegarde
        proficiency_bonus = calculate_proficiency(rank, level)
        ability_mod = atrs_char[ability]["mod"]
        total_mod = ability_mod + proficiency_bonus

        # Remplir les champs
        all_fields_dict[f"{save_pdf}"] = total_mod
        all_fields_dict[f"{save_pdf}_QUALIFIE"] = "/Oui" if rank >= 1 else "/Off"
        all_fields_dict[f"{save_pdf}_EXPERT"] = "/Oui" if rank >= 2 else "/Off"
        all_fields_dict[f"{save_pdf}_MAITRE"] = "/Oui" if rank >= 3 else "/Off"
        all_fields_dict[f"{save_pdf}_LEGENDAIRE"] = "/Oui" if rank >= 4 else "/Off"
        all_fields_dict[f"{save_pdf}_{ATTRIBUTES[ability]}"] = ability_mod
        all_fields_dict[f"{save_pdf}_MAITRISE"] = proficiency_bonus

    # Perception
    perception_rank = proficiency_rank["perception"]
    perception_proficiency = calculate_proficiency(perception_rank, level)
    perception_mod = atrs_char["wis"]["mod"] + perception_proficiency

    all_fields_dict["PERCEPTION"] = perception_mod
    all_fields_dict["PERCEPTION_QUALIFIE"] = "/Oui" if perception_rank >= 1 else "/Off"
    all_fields_dict["PERCEPTION_EXPERT"] = "/Oui" if perception_rank >= 2 else "/Off"
    all_fields_dict["PERCEPTION_MAITRE"] = "/Oui" if perception_rank >= 3 else "/Off"
    all_fields_dict["PERCEPTION_LEGENDAIRE"] = (
        "/Oui" if perception_rank >= 4 else "/Off"
    )
    all_fields_dict["PERCEPTION_SAGESSE"] = atrs_char["wis"]["mod"]
    all_fields_dict["PERCEPTION_MAITRISE"] = perception_proficiency

    # Classe d'armure (CA)
    # Le PDF contient des champs spécifiques pour chaque type d'armure
    armor_types = {
        "unarmored": "SANS_ARMURE",
        "light": "ARMURE_LEGERE",
        "medium": "ARMURE_MOYENNE",
        "heavy": "ARMURE_LOURDE",
    }

    # Remplir les cases pour chaque type d'armure
    for armor_type in armor_types.keys():
        all_fields_dict[f"{armor_types[armor_type]}_QUALIFIE"] = (
            "/Oui" if proficiency_rank[armor_type] >= 1 else "/Off"
        )
        all_fields_dict[f"{armor_types[armor_type]}_EXPERT"] = (
            "/Oui" if proficiency_rank[armor_type] >= 2 else "/Off"
        )
        all_fields_dict[f"{armor_types[armor_type]}_MAITRE"] = (
            "/Oui" if proficiency_rank[armor_type] >= 3 else "/Off"
        )
        all_fields_dict[f"{armor_types[armor_type]}_LEGENDAIRE"] = (
            "/Oui" if proficiency_rank[armor_type] >= 4 else "/Off"
        )

    # Déterminer le type d'armure porté (à implémenter plus tard)
    # Pour l'instant, on utilise le type avec la maîtrise la plus élevée
    armor_bonus, dexCap = 0, 6
    if list_don_by_categ(character_data, "armor"):
        worn_armor_type = next_item_by_type(character_data, "armor")["system"][
            "category"
        ]
        armor_rank = proficiency_rank[worn_armor_type]
        armor_bonus = next_item_by_type(character_data, "armor")["system"].get(
            "acBonus", 0
        )
        dexCap = next_item_by_type(character_data, "armor")["system"].get("dexCap", 6)

    # Calculer le bonus de maîtrise
    ac_proficiency = calculate_proficiency(armor_rank, level)

    # Utiliser le modificateur de DEX
    dex_mod = min(atrs_char["dex"]["mod"], dexCap)

    # Calcul final de la CA
    base_ac = 10 + dex_mod + ac_proficiency + armor_bonus

    # Remplir les champs principaux de la CA
    all_fields_dict["CLASSE_ARMURE"] = base_ac
    all_fields_dict["CLASSE_ARMURE_DEX"] = dex_mod
    all_fields_dict["CLASSE_ARMURE_MAITRISE"] = ac_proficiency
    all_fields_dict["CLASSE_ARMURE_OBJET"] = armor_bonus

    # Langues
    # Dans la fonction prepare_pdf_fields, remplacer la partie des langues par:
    language_names = get_all_languages(character_data)
    if language_names:
        # Joindre toutes les langues en une seule chaîne séparée par des virgules
        all_languages = ", ".join(language_names)

        # Remplir le champ approprié dans le PDF
        all_fields_dict["LANGUES"] = all_languages

    # Sorts
    if proficiency_rank.get("spellcasting", False):
        prepare_spells_fields(
            all_fields_dict,
            character_data,
            proficiency_rank["spellcasting"],
            abilities=atrs_char,
        )

    # Actions
    actions = list_don_by_categ(character, "action")

    for i, action in enumerate(actions, start=1):
        if i > 6:
            break
        action_name = action["name"]
        action_value = action["system"]["actions"]["value"]
        action_traits = action["system"]["traits"]["value"]
        action_description = description_text_cleaner(
            action["system"]["description"]["value"]
        )

        all_fields_dict[f"ACTION_ACTIVITE_{i}_NOM"] = action_name
        all_fields_dict[f"ACTION_ACTIVITE_{i}_ACTIONS"] = action_value
        all_fields_dict[f"ACTION_ACTIVITE_{i}_TRAITS"] = ", ".join(action_traits)

    # Arme
    weapons = list_don_by_categ(character, "weapon")
    nb_cac, nb_distance = 0, 0
    for weapon in weapons:
        weapon_name = weapon["name"]
        weapon_damage_type = weapon["system"]["damage"]["damageType"]
        weapon_range = weapon["system"]["range"]
        weapon_damage = (
            str(weapon["system"]["damage"]["dice"]) + weapon["system"]["damage"]["die"]
        )
        weapon_description = f"range: {weapon_range}, traits: " + ", ".join(
            weapon["system"]["traits"]["value"]
        )
        weapon_force = atrs_char["str"]["mod"]
        weapon_dex = atrs_char["dex"]["mod"]
        weapon_proficiency = calculate_proficiency(
            proficiency_rank.get(weapon["system"]["category"], 0), level
        )
        weapon_objet = weapon["system"]["bonus"]["value"]
        if weapon["system"].get("range", 0) <= 15:
            nb_cac += 1
            all_fields_dict[f"ARME_CAC_{nb_cac}"] = weapon_name
            all_fields_dict[f"ARME_CAC_{nb_cac}_DEGATS"] = weapon_damage
            all_fields_dict[f"ARME_CAC_{nb_cac}_RANGE"] = weapon_range
            all_fields_dict[f"ARME_CAC_{nb_cac}_TRAITS_NOTES"] = weapon_description
            all_fields_dict[f"ARME_CAC_{nb_cac}_BONUS_ATTAQUE"] = (
                weapon_force + weapon_proficiency + weapon_objet
            )
            all_fields_dict[f"ARME_CAC_{nb_cac}_FORCE"] = weapon_force
            all_fields_dict[f"ARME_CAC_{nb_cac}_MAITRISE"] = weapon_proficiency
            all_fields_dict[f"ARME_CAC_{nb_cac}_OBJET"] = weapon_objet
            match weapon_damage_type:
                case "bludgeoning":
                    all_fields_dict[f"ARME_CAC_{nb_cac}_CONTONDANT"] = "/Oui"
                case "piercing":
                    all_fields_dict[f"ARME_CAC_{nb_cac}_PERFORANT"] = "/Oui"
                case "slashing":
                    all_fields_dict[f"ARME_CAC_{nb_cac}_TRANCHANT"] = "/Oui"
        else:
            nb_distance += 1
            all_fields_dict[f"ARME_DIST_{nb_distance}"] = weapon_name
            all_fields_dict[f"ARME_DIST_{nb_distance}_DEGATS"] = weapon_damage
            all_fields_dict[f"ARME_DIST_{nb_distance}_RANGE"] = weapon_range
            all_fields_dict[f"ARME_DIST_{nb_distance}_TRAITS_NOTES"] = (
                weapon_description
            )
            all_fields_dict[f"ARME_DIST_{nb_distance}_BONUS_ATTAQUE"] = (
                weapon_dex + weapon_proficiency + weapon_objet
            )
            all_fields_dict[f"ARME_DIST_{nb_distance}_FORCE"] = weapon_dex
            all_fields_dict[f"ARME_DIST_{nb_distance}_MAITRISE"] = weapon_proficiency
            all_fields_dict[f"ARME_DIST_{nb_distance}_OBJET"] = weapon_objet
            match weapon_damage_type:
                case "bludgeoning":
                    all_fields_dict[f"ARME_DIST_{nb_distance}_CONTONDANT"] = "/Oui"
                case "piercing":
                    all_fields_dict[f"ARME_DIST_{nb_distance}_PERFORANT"] = "/Oui"
                case "slashing":
                    all_fields_dict[f"ARME_DIST_{nb_distance}_TRANCHANT"] = "/Oui"


prepare_pdf_fields(all_fields_dict, character)

In [8]:
# Création du dictionnaire de champs modifier
replace_field_dict = {
    key: value for key, value in all_fields_dict.items() if value is not None
}

In [9]:
# Remplacement des champs que nous avons modifiés
writer.update_page_form_field_values(None, replace_field_dict)
# Sauvegarde du PDF
save_pdf(writer, output_path)

### Abilities

In [10]:
build_abilities(character)

{'str': {'value': 8, 'mod': -1, 'partial': False},
 'dex': {'value': 18, 'mod': 4, 'partial': True},
 'con': {'value': 16, 'mod': 3, 'partial': False},
 'int': {'value': 18, 'mod': 4, 'partial': False},
 'wis': {'value': 10, 'mod': 0, 'partial': False},
 'cha': {'value': 12, 'mod': 1, 'partial': False}}

### Compétences

In [11]:
build_competences(character, 2, abilities=build_abilities(character))

{'acrobatics': {'rank': 1,
  'mod': 8,
  'ability': 'dex',
  'proficiency_bonus': 4,
  'ability_mod': 4},
 'arcana': {'rank': 1,
  'mod': 8,
  'ability': 'int',
  'proficiency_bonus': 4,
  'ability_mod': 4},
 'athletics': {'rank': 0,
  'mod': -1,
  'ability': 'str',
  'proficiency_bonus': 0,
  'ability_mod': -1},
 'crafting': {'rank': 2,
  'mod': 10,
  'ability': 'int',
  'proficiency_bonus': 6,
  'ability_mod': 4},
 'deception': {'rank': 1,
  'mod': 5,
  'ability': 'cha',
  'proficiency_bonus': 4,
  'ability_mod': 1},
 'diplomacy': {'rank': 1,
  'mod': 5,
  'ability': 'cha',
  'proficiency_bonus': 4,
  'ability_mod': 1},
 'intimidation': {'rank': 0,
  'mod': 1,
  'ability': 'cha',
  'proficiency_bonus': 0,
  'ability_mod': 1},
 'medicine': {'rank': 0,
  'mod': 0,
  'ability': 'wis',
  'proficiency_bonus': 0,
  'ability_mod': 0},
 'nature': {'rank': 0,
  'mod': 0,
  'ability': 'wis',
  'proficiency_bonus': 0,
  'ability_mod': 0},
 'occultism': {'rank': 0,
  'mod': 4,
  'ability': 'int'

### Édition du livre de sors

In [50]:
def text_cleaner(text):
    text = re.sub(r"@\w+\[.*?\]{(.+?)}", r"<strong>\1</strong>", text)
    text = re.sub(
        r"@(\w+)\[(.*)\|(.*)\|.*\]",
        r"<strong>\1 \2 \3</strong>",
        text,
    )
    return text


def generate_character_pages_html(character_data):
    """
    Génère une représentation HTML du grimoire de sorts, de l'inventaire et de la liste des dons.

    Args:
        character_data (dict): Les données du personnage au format JSON

    Returns:
        str: Le code HTML généré
    """
    # CSS pour la mise en page et le style
    css = """
    .page {
        padding: 20px;
        box-sizing: border-box;
        width: 21cm;
        margin: 0 auto;
        background: #fff;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.2);
        display: none; /* Cacher toutes les pages par défaut */
    }
    .page.active {
        display: block; /* Afficher uniquement la page active */
    }
    .page-header {
        text-align: center;
        border-bottom: 2px solid #5E0000;
        padding-bottom: 10px;
        margin-bottom: 20px;
    }
    h1, h2, h3, h4 {
        font-family: 'Pathfinder', 'Times New Roman', serif;
        margin-top: 0;
        color: #5E0000;
    }
    h1 {
        font-size: 24pt;
        margin-bottom: 10px;
    }
    h2 {
        font-size: 18pt;
        border-bottom: 1px solid #5E0000;
        padding-bottom: 5px;
        margin-top: 20px;
    }
    h3 {
        font-size: 14pt;
        margin-bottom: 5px;
    }
    .section-title {
        font-size: 18pt;
        border-bottom: 1px solid #5E0000;
        padding-bottom: 5px;
        margin-top: 0px;
        column-span: all; /* L'élément s'étend sur toutes les colonnes */
    }
    .section {
        columns: 2;
        gap: 10px;
        background-color: #F0E6D2;
        border: 1px solid #D4C8B0;
        border-radius: 5px;
        padding: 10px;
        margin-bottom: 15px;
        width: calc(100% - 20px);
        break-inside: avoid-page;
    }
    .section-item-unique {
        gap: 10px;
        background-color: #F0E6D2;
        border: 1px solid #D4C8B0;
        border-radius: 5px;
        padding: 10px;
        margin-bottom: 15px;
        width: calc(100% - 20px);
        break-inside: avoid-page;
    }
    .item {
        margin-bottom: 10px;
        background-color: #FFFFFF;
        border: 1px solid #E0D8C0;
        border-radius: 5px;
        padding: 10px;
        box-sizing: border-box;
        break-inside: avoid-column;
    }
    .item-long {
        margin-bottom: 10px;
        background-color: #FFFFFF;
        border: 1px solid #E0D8C0;
        border-radius: 5px;
        padding: 10px;
        box-sizing: border-box;
    }
    /* Cacher les URL et autres informations ajoutées automatiquement */
    :root {
        -webkit-print-color-adjust: exact !important;
        print-color-adjust: exact !important;
    }
    .item-header {
        display: flex;
        justify-content: space-between;
        border-bottom: 1px solid #E0D8C0;
        padding-bottom: 5px;
        margin-bottom: 5px;
        font-weight: bold;
    }
    .item-traits {
        margin-top: 5px;
        margin-bottom: 5px;
    }
    .trait {
        display: inline-block;
        background-color: #F0E6D2;
        border: 1px solid #D4C8B0;
        border-radius: 3px;
        padding: 2px 5px;
        margin-right: 5px;
        font-size: 9pt;
        color: #5E0000;
    }
    .item-description {
        margin-top: 8px;
        font-size: 10pt;
    }
    .metadata {
        display: flex;
        flex-wrap: wrap;
        gap: 10px;
        font-size: 10pt;
        color: #666;
        margin-top: 5px;
    }
    .meta-item {
        margin-right: 10px;
    }
    .actions {
        color: #5E0000;
        font-weight: bold;
    }
    
    /* Styles pour la barre de navigation */
    .nav-bar {
        position: fixed;
        top: 0;
        left: 0;
        width: 100%;
        background-color: #5E0000;
        color: white;
        padding: 10px 0;
        box-shadow: 0 2px 5px rgba(0, 0, 0, 0.2);
        z-index: 100;
        display: flex;
        justify-content: center;
        align-items: center;
    }
    .nav-bar a {
        color: white;
        text-decoration: none;
        padding: 8px 16px;
        margin: 0 5px;
        border-radius: 3px;
        transition: background-color 0.3s;
    }
    .nav-bar a:hover {
        background-color: #7E2020;
    }
    .nav-bar a.active {
        background-color: #7E2020;
        font-weight: bold;
    }
    .content {
        margin-top: 60px; /* Espace pour la barre de navigation */
        padding-bottom: 20px;
    }
    
    @media print {
        body {
            background: none;
            margin: 0;
            padding: 0;
        }
        .page {
            display: block !important; /* Afficher toutes les pages pour l'impression */
            box-shadow: none;
            margin: 0;
            width: 100%;
            height: auto;
            page-break-after: always;
        }
        .nav-bar {
            display: none; /* Cacher la barre de navigation pour l'impression */
        }
        .content {
            margin-top: 0;
        }
    }
    @page {
        margin: 0.5cm;
        size: A4;
        /* Supprimer les en-têtes et pieds de page par défaut du navigateur */
        margin-header: 0;
        margin-footer: 0;
    }
    """

    # JavaScript pour la navigation
    javascript = """
    document.addEventListener('DOMContentLoaded', function() {
        // Fonction pour afficher une page et masquer les autres
        function showPage(pageId) {
            // Masquer toutes les pages
            document.querySelectorAll('.page').forEach(function(page) {
                page.classList.remove('active');
            });
            
            // Afficher la page sélectionnée
            document.getElementById(pageId).classList.add('active');
            
            // Mettre à jour les liens actifs dans la barre de navigation
            document.querySelectorAll('.nav-bar a').forEach(function(link) {
                if (link.getAttribute('data-page') === pageId) {
                    link.classList.add('active');
                } else {
                    link.classList.remove('active');
                }
            });
        }
        
        // Ajouter des écouteurs d'événements pour les liens de navigation
        document.querySelectorAll('.nav-bar a').forEach(function(link) {
            link.addEventListener('click', function(e) {
                e.preventDefault();
                showPage(this.getAttribute('data-page'));
            });
        });
        
        // Afficher la première page par défaut
        showPage('spellbook');
    });
    """

    # Structure HTML de base
    character_name = character_data["name"]
    html = f"""<!DOCTYPE html>
    <html lang="fr">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Personnage - {character_name}</title>
        <style>{css}</style>
    </head>
    <body>
        <!-- Barre de navigation -->
        <div class="nav-bar">
            <a href="#" data-page="spellbook" class="active">Grimoire</a>
            <a href="#" data-page="inventory">Inventaire</a>
            <a href="#" data-page="feats">Dons</a>
        </div>
        
        <!-- Conteneur principal -->
        <div class="content">
    """

    # Génération des différentes pages
    html += generate_spellbook_page(character_data, page_id="spellbook")
    html += generate_inventory_page(character_data, page_id="inventory")
    html += generate_feats_page(character_data, page_id="feats")

    html += (
        """
        </div>
        <script>"""
        + javascript
        + """</script>
    </body>
    </html>
    """
    )
    return html


def generate_spellbook_page(character_data, page_id="spellbook"):
    """Génère la page de grimoire"""
    spellbook = build_spellbook(character_data)

    character_name = character_data["name"]
    character_level = character_data["system"]["details"]["level"]["value"]
    character_class = next(
        (item["name"] for item in character_data["items"] if item["type"] == "class"),
        "",
    )

    html = f"""
    <div id="{page_id}" class="page active">
        <div class="page-header">
            <h1>Grimoire de {character_name}</h1>
            <h3>Niveau {character_level} {character_class}</h3>
        </div>
    """

    # Tours de magie
    if spellbook["cantrips"]:
        html += f"""
        <div class={"section" if len(spellbook["cantrips"]) > 1 else "section-item-unique"}>
        <h2 class="section-title">Tours de magie</h2>
        """
        for spell in spellbook["cantrips"]:
            html += format_spell_html(spell)
        html += "</div>"

    # Sorts focalisés
    if spellbook["focus"]:
        html += f"""
        <div class={"section" if len(spellbook["focus"]) > 1 else "section-item-unique"}>
        <h2 class="section-title">Sorts focalisés</h2>
        """
        for spell in spellbook["focus"]:
            html += format_spell_html(spell)
        html += "</div>"

    # Sorts par niveau
    for level, spells in spellbook["spells"].items():
        if spells:
            html += f"""
            <div class={"section" if len(spells) > 1 else "section-item-unique"}>
            <h2 class="section-title">Sorts de niveau {level}</h2>
            """
            for spell in spells:
                html += format_spell_html(spell)
            html += "</div>"

    html += "</div>"  # Fin de la page
    return html


def generate_feats_page(character_data, page_id="feats"):
    """Génère la page de dons"""
    character_name = character_data["name"]
    character_level = character_data["system"]["details"]["level"]["value"]
    character_class = next(
        (item["name"] for item in character_data["items"] if item["type"] == "class"),
        "",
    )

    html = f"""
    <div id="{page_id}" class="page">
        <div class="page-header">
            <h1>Dons et capacités de {character_name}</h1>
            <h3>Niveau {character_level} {character_class}</h3>
        </div>
    """

    # Récupérer tous les dons
    all_feats = list_don_by_categ(character_data, "feat")

    # Organiser les dons par catégorie
    feat_categories = {}
    for feat in all_feats:
        category = feat["system"].get("category", "other")
        if category not in feat_categories:
            feat_categories[category] = []
        feat_categories[category].append(feat)

    # Traduire les noms de catégories
    category_translations = {
        "ancestry": "Dons d'ascendance",
        "class": "Dons de classe",
        "skill": "Dons de compétence",
        "general": "Dons généraux",
        "archetype": "Dons d'archétype",
        "other": "Autres capacités",
    }

    # Ordre des catégories
    category_order = ["ancestry", "class", "archetype", "skill", "general", "other"]

    # Afficher les dons par catégorie dans l'ordre défini
    for category in category_order:
        if category in feat_categories and feat_categories[category]:
            html += f"""
            <div class={"section" if len(feat_categories[category]) > 1 else "section-item-unique"}>
            <h2 class="section-title">{category_translations.get(category, category.capitalize())}</h2>
            """
            for feat in feat_categories[category]:
                html += format_feat_html(feat)
            html += "</div>"

    # Afficher les autres catégories qui ne sont pas dans l'ordre prédéfini
    for category, feats in feat_categories.items():
        if category not in category_order:
            html += f"""
            <div class={"section" if len(feats) > 1 else "section-item-unique"}>
            <h2 class="section-title">{category_translations.get(category, category.capitalize())}</h2>
            """
            for feat in feats:
                html += format_feat_html(feat)
            html += "</div>"

    html += "</div>"  # Fin de la page
    return html


def generate_inventory_page(character_data, page_id="inventory"):
    """Génère la page d'inventaire"""
    character_name = character_data["name"]
    character_level = character_data["system"]["details"]["level"]["value"]
    character_class = next(
        (item["name"] for item in character_data["items"] if item["type"] == "class"),
        "",
    )

    html = f"""
    <div id="{page_id}" class="page">
        <div class="page-header">
            <h1>Inventaire de {character_name}</h1>
            <h3>Niveau {character_level} {character_class}</h3>
        </div>
    """

    # Catégories d'inventaire
    categories = {
        "weapon": "Armes",
        "armor": "Armures",
        "equipment": "Équipement",
        "consumable": "Consommables",
        "treasure": "Trésors",
    }

    for category_key, category_name in categories.items():
        items = list_don_by_categ(character_data, category_key)
        if items:
            html += f"""
            <div class={"section" if len(items) > 1 else "section-item-unique"}>
            <h2 class="section-title">{category_name}</h2>
            """
            for item in items:
                html += format_item_html(item)
            html += "</div>"

    html += "</div>"  # Fin de la page
    return html


def format_spell_html(spell):
    """Formate un sort en HTML"""
    name = spell.name if hasattr(spell, "name") else "Sort sans nom"
    level = spell.level if hasattr(spell, "level") else 0
    actions = spell.actions if hasattr(spell, "actions") else ""
    traits = spell.traits if hasattr(spell, "traits") else []
    description = text_cleaner(
        spell.description if hasattr(spell, "description") else ""
    )

    # Nettoyer la description pour éviter les problèmes HTML
    description = description.replace("<p>", "").replace("</p>", "<br>")

    html = f"""
    <div class="{"item-long" if len(description) > 2000 else "item"}">
        <div class="item-header">
            <div>{name}</div>
            <div class="actions">{actions if actions else "—"}</div>
        </div>
    """

    if traits:
        html += '<div class="item-traits">'
        for trait in traits:
            html += f'<span class="trait">{trait}</span>'
        html += "</div>"

    if description:
        html += f'<div class="item-description">{description}</div>'

    html += "</div>"  # Fermeture de l'item
    return html


def format_item_html(item):
    """Formate un objet d'inventaire en HTML"""
    name = item["name"]
    description = ""
    if item["system"].get("description"):
        description = text_cleaner(item["system"]["description"].get("value", ""))

    traits = []
    if item["system"].get("traits"):
        traits = item["system"]["traits"].get("value", [])

    html = f"""
    <div class="{"item-long" if len(description) > 2000 else "item"}">
        <div class="item-header">
            <div>{name}</div>
    """

    # Informations spécifiques selon le type d'objet
    if item["type"] == "weapon":
        damage_dice = item["system"].get("damage", {}).get("dice", "")
        damage_die = item["system"].get("damage", {}).get("die", "")
        damage_type = item["system"].get("damage", {}).get("damageType", "")
        html += f"<div>{damage_dice}{damage_die} {damage_type}</div>"
    elif item["type"] == "armor":
        ac_bonus = item["system"].get("acBonus", 0)
        html += f"<div>CA +{ac_bonus}</div>"

    html += "</div>"  # Fermeture de item-header

    if traits:
        html += '<div class="item-traits">'
        for trait in traits:
            html += f'<span class="trait">{trait}</span>'
        html += "</div>"

    # Métadonnées
    html += '<div class="metadata">'

    # Informations supplémentaires selon le type d'objet
    if item["type"] == "weapon":
        weapon_range = item["system"].get("range", 0)
        html += f'<span class="meta-item">Portée: {weapon_range}</span>'
    elif item["type"] == "armor":
        dex_cap = item["system"].get("dexCap", 0)
        html += f'<span class="meta-item">Limite Dex: {dex_cap}</span>'

    bulk = item["system"].get("bulk", {}).get("value", "L")
    html += f'<span class="meta-item">Encombrement: {bulk}</span>'

    html += "</div>"  # Fermeture de metadata

    if description:
        html += f'<div class="item-description">{description}</div>'

    html += "</div>"  # Fermeture de item
    return html


def format_feat_html(feat):
    """Formate un don en HTML"""
    name = feat["name"]
    level = feat["system"].get("level", {}).get("value", "")

    description = ""
    if feat["system"].get("description"):
        description = text_cleaner(feat["system"]["description"].get("value", ""))

    traits = []
    if feat["system"].get("traits"):
        traits = feat["system"]["traits"].get("value", [])

    html = f"""
    <div class="{"item-long" if len(description) > 2000 else "item"}">
        <div class="item-header">
            <div>{name}</div>
            <div>Niveau {level}</div>
        </div>
    """

    if traits:
        html += '<div class="item-traits">'
        for trait in traits:
            html += f'<span class="trait">{trait}</span>'
        html += "</div>"

    # Prérequis
    prerequisites = feat["system"].get("prerequisites", {}).get("value", [])
    if prerequisites:
        prereq_text = ", ".join(
            p.get("value", "") for p in prerequisites if p.get("value")
        )
        if prereq_text:
            html += f'<div class="metadata"><span class="meta-item">Prérequis: {prereq_text}</span></div>'

    if description:
        html += f'<div class="item-description">{description}</div>'

    html += "</div>"  # Fermeture de item
    return html

In [51]:
# Charger les données du personnage
character = load_character_data(json_path)

# Générer le HTML
html_content = generate_character_pages_html(character)

# Enregistrer dans un fichier HTML
with open("personnage.html", "w", encoding="utf-8") as f:
    f.write(html_content)

# Analyse en profondeur des données

In [14]:
# comptage des items par types
item_types_count = {}
for item in [item for item in character["items"]]:
    item_type = item["type"]
    if item_type in item_types_count:
        item_types_count[item_type] += 1
    else:
        item_types_count[item_type] = 1
pprint(item_types_count)

{'action': 2,
 'ancestry': 1,
 'armor': 1,
 'background': 1,
 'class': 1,
 'deity': 1,
 'feat': 15,
 'heritage': 1,
 'lore': 1,
 'spell': 17,
 'spellcastingEntry': 2,
 'weapon': 1}


In [15]:
# catégories des feats
feat_types = []
for feat_ in list_don_by_categ(character, "feat"):
    if feat_["system"]["category"] not in feat_types:
        feat_types.append(feat_["system"]["category"])
feat_types

['ancestryfeature', 'classfeature', 'ancestry', 'skill', 'class']

In [16]:
# data dans le character
character.keys()

dict_keys(['prototypeToken', 'name', 'type', 'effects', 'system', 'img', 'items', 'folder', 'flags', '_stats'])

In [17]:
# count des items par clefs
keys_item_count = {}
for item in character["items"]:
    for key in item.keys():
        keys_item_count[key] = keys_item_count.get(key, 0) + 1
keys_item_count

{'_id': 44,
 'img': 44,
 'name': 44,
 'system': 44,
 'type': 44,
 '_stats': 44,
 'effects': 44,
 'folder': 44,
 'sort': 44,
 'flags': 44}

In [18]:
# analyse des items en fonction de leurs clefs
def analyze_items(character, item_keys):
    return [
        (item_["name"], item_["type"])
        for item_ in character["items"]
        if item_["system"].get(item_keys, False)
    ]


analyze_items(character, "hp")

[('Sprite', 'ancestry'),
 ('Magus', 'class'),
 ('Pistolet de duel (Dueling Pistol)', 'weapon'),
 ("Vêtements d'explorateur (Explorer's Clothing)", 'armor')]

In [19]:
# analyse des flags
flags_keys = set()
for i in character["items"]:
    if i["flags"]:
        flags_keys.update(i["flags"].keys())
flags_keys

{'babele', 'pf2e'}

In [20]:
# Création d'une fonction Pour identifier les tags système De chaque type d'item
# { ancestry: [description, rules, ...],
#   background: [description, rules, ...],
#   class: [description, rules, ...],
#   ...
# }
def identify_system_tags(character_data):
    """Identifie les tags système pour chaque type d'item dans les données du personnage."""
    system_tags = {}
    # Parcours des items du personnage
    for item in character_data["items"]:
        # Récupération du type d'item
        item_type = item["type"]
        # Vérification de l'existence du type dans les tags système
        if item_type not in system_tags:
            # Initialisation de la liste des tags pour ce type
            system_tags[item_type] = set()
        # Ajout des tags système de l'item à la liste correspondante
        system_tags[item_type].update(item["system"].keys())
    return system_tags


identify_system_tags(character)

{'ancestry': {'_migration',
  'additionalLanguages',
  'boosts',
  'description',
  'flaws',
  'hp',
  'items',
  'languages',
  'publication',
  'reach',
  'rules',
  'size',
  'slug',
  'speed',
  'traits',
  'vision',
  'voluntary'},
 'feat': {'_migration',
  'actionType',
  'actions',
  'category',
  'description',
  'frequency',
  'level',
  'location',
  'maxTakable',
  'onlyLevel1',
  'prerequisites',
  'publication',
  'rules',
  'slug',
  'subfeatures',
  'traits'},
 'heritage': {'_migration',
  'ancestry',
  'description',
  'publication',
  'rules',
  'slug',
  'traits'},
 'class': {'_migration',
  'ancestryFeatLevels',
  'attacks',
  'classFeatLevels',
  'defenses',
  'description',
  'generalFeatLevels',
  'hp',
  'items',
  'keyAbility',
  'perception',
  'publication',
  'rules',
  'savingThrows',
  'skillFeatLevels',
  'skillIncreaseLevels',
  'slug',
  'spellcasting',
  'trainedSkills',
  'traits'},
 'action': {'_migration',
  'actionType',
  'actions',
  'category',
 

In [21]:
import pandas as pd

memory = pd.DataFrame()
for item_type, tags in identify_system_tags(character).items():
    memory = pd.concat(
        [memory, pd.DataFrame(True, columns=list(tags), index=[item_type])]
    )
memory

,languages,rules,flaws,publication,traits,size,voluntary,description,items,additionalLanguages,...,defense,duration,overlays,cost,time,counteraction,area,requirements,heightening,target
ancestry,True,True,True,True,True,True,True,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
feat,NaN,True,NaN,True,True,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
heritage,NaN,True,NaN,True,True,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
class,NaN,True,NaN,True,True,NaN,NaN,True,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
action,NaN,True,NaN,True,True,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
background,NaN,True,NaN,True,True,NaN,NaN,True,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weapon,NaN,True,NaN,True,True,True,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
deity,NaN,True,NaN,True,True,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lore,NaN,True,NaN,True,True,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
armor,NaN,True,NaN,True,True,True,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
keys_item_sys_count = {}
for item in character["items"]:
    for item_sys_key in item["system"].keys():
        keys_item_sys_count[item_sys_key] = keys_item_sys_count.get(item_sys_key, 0) + 1
keys_item_sys_count

{'description': 44,
 'rules': 44,
 'slug': 44,
 '_migration': 44,
 'traits': 44,
 'publication': 44,
 'hp': 4,
 'size': 3,
 'reach': 1,
 'speed': 1,
 'boosts': 2,
 'flaws': 1,
 'languages': 1,
 'additionalLanguages': 1,
 'items': 3,
 'vision': 1,
 'voluntary': 1,
 'actionType': 17,
 'actions': 17,
 'category': 20,
 'level': 34,
 'prerequisites': 15,
 'onlyLevel1': 15,
 'maxTakable': 15,
 'subfeatures': 15,
 'location': 32,
 'ancestry': 1,
 'ancestryFeatLevels': 1,
 'attacks': 1,
 'classFeatLevels': 1,
 'defenses': 1,
 'generalFeatLevels': 1,
 'keyAbility': 1,
 'perception': 1,
 'savingThrows': 1,
 'skillFeatLevels': 1,
 'skillIncreaseLevels': 1,
 'spellcasting': 1,
 'trainedSkills': 2,
 'selfEffect': 1,
 'frequency': 1,
 'quantity': 2,
 'baseItem': 2,
 'bulk': 2,
 'hardness': 2,
 'price': 2,
 'equipped': 2,
 'containerId': 2,
 'material': 2,
 'identification': 2,
 'usage': 1,
 'group': 2,
 'bonus': 1,
 'damage': 18,
 'splashDamage': 1,
 'range': 18,
 'expend': 1,
 'reload': 1,
 'runes'